In [1]:
#Import libraries and load Data
import pandas as pd

data = pd.read_csv('/home/siddhant/Stout-Case-Study/CaseStudy2/casestudy.csv')

#Use this code to load data
#data = pd.read_csv('./casestudy.csv')

data.head(5)

,Unnamed: 0,customer_email,net_revenue,year
0,0,nhknapwsbx@gmail.com,249.92,2015
1,1,joiuzbvcpn@gmail.com,87.61,2015
2,2,ukkjctepxt@gmail.com,168.38,2015
3,3,gykatilzrt@gmail.com,62.40,2015
4,4,mmsgsrtxah@gmail.com,43.08,2015


In [2]:
#Find number of rows
len(data)

685927

In [3]:
#Finding data types
data.dtypes

Unnamed: 0          int64
customer_email     object
net_revenue       float64
year                int64
dtype: object

# Total revenue for the current year

In [4]:
totalRevenueData = data.groupby('year').agg(total_net_revenue=('net_revenue', 'sum'))
totalRevenueData

,total_net_revenue
year,
2015,29036749.19
2016,25730943.59
2017,31417495.03


# New Customer Revenue e.g. new customers not present in previous year only

In [5]:
newCustomerRevenueData = pd.merge(data, data, on='customer_email', how='left')
newCustomerRevenueData.drop(['Unnamed: 0_y','net_revenue_y'], axis=1, inplace=True)
newCustomerRevenueData = newCustomerRevenueData.query('year_x == year_y+1')
newCustomerRevenueData.groupby('year_x').agg(new_customer_revenue=('net_revenue_x', 'sum'))

,new_customer_revenue
year_x,
2016,7485452.58
2017,2641259.99


# Existing Customer Growth. To calculate this, use the Revenue of existing customers for current year –(minus) Revenue of existing customers from the previous year

In [6]:
dataECG = data.groupby('year').agg(total_revenue=('net_revenue', 'sum'))
dataECG['year'] = dataECG.index
dataECG.reset_index(drop=True)
dataECG['key'] = 1
dataECG = pd.merge(dataECG, dataECG, on='key').drop('key', 1)
dataECG = dataECG.query('year_x == year_y+1')
dataECG['existing_customer_growth'] = dataECG['total_revenue_x']-dataECG['total_revenue_y']
dataECG.drop(['total_revenue_x','total_revenue_y','year_y'], axis=1, inplace=True)
dataECG

,year_x,existing_customer_growth
3,2016,-3305805.60
7,2017,5686551.44


# Revenue lost from attrition

In [7]:
ECGData = data.groupby('year').agg(total_revenue=('net_revenue', 'sum'))
ECGData['year'] = ECGData.index
ECGData.reset_index(drop=True)
ECGData['key'] = 1
ECGData = pd.merge(ECGData, ECGData, on='key').drop('key', 1)
ECGData = ECGData.query('year_x == year_y+1')
ECGData['revenue_lost'] = ECGData['total_revenue_y']-ECGData['total_revenue_x']
ECGData.drop(['total_revenue_x','total_revenue_y','year_x'], axis=1, inplace=True)
ECGData

,year_y,revenue_lost
3,2015,3305805.60
7,2016,-5686551.44


# Existing Customer Revenue Current Year

In [8]:
totalRevenueData

,total_net_revenue
year,
2015,29036749.19
2016,25730943.59
2017,31417495.03


# Existing Customer Revenue Prior Year


In [9]:
totalRevenueData['year'] = totalRevenueData.index
totalRevenueData.reset_index(drop=True)
totalRevenueData['key'] = 1
ECRPriorYearData = pd.merge(totalRevenueData, totalRevenueData, on='key').drop('key', 1)
ECRPriorYearData = ECRPriorYearData.query('year_x == year_y+1')
ECRPriorYearData.drop(['total_net_revenue_x','year_y'], axis=1, inplace=True)
ECRPriorYearData

,year_x,total_net_revenue_y
3,2016,29036749.19
7,2017,25730943.59


# New Customers

In [10]:
yearsList = data['year'].unique().tolist()
yearsList.sort()

customers = {}

newCustomers = {}
lostCustomers = {}

for presentYear in yearsList[1:]:
    
    #Finding present year customers
    if presentYear not in customers:
        currentYearCustomers = data['customer_email'].loc[data['year'] == presentYear].tolist()
        customers[presentYear] = currentYearCustomers
    else:
        currentYearCustomers = customers[presentYear]
    
    #Finding previous years customers
    if presentYear-1 not in customers:
        previousYearCustomers = data['customer_email'].loc[data['year'] == presentYear-1].tolist()
        customers[presentYear-1] = previousYearCustomers
    else:
        previousYearCustomers = customers[presentYear-1]
    
    newCustomers[presentYear] = list(set(currentYearCustomers) - set(previousYearCustomers))
    lostCustomers[presentYear] = list(set(previousYearCustomers) - set(currentYearCustomers))

newCustomerData = pd.DataFrame(newCustomers.items(), columns=['Year', 'New_Customers'])
newCustomerData['New_Customers_Count'] = newCustomerData['New_Customers'].str.len()
newCustomerData

,Year,New_Customers,New_Customers_Count
0,2016,"[rcvcbonbsr@gmail.com, qiopuufmpc@gmail.com, ...",145062
1,2017,"[uvxrfjeqby@gmail.com, fbwelaprmw@gmail.com, d...",229028


# Lost Customers

In [11]:
lostCustomerData = pd.DataFrame(lostCustomers.items(), columns=['Year', 'Lost_Customers'])
lostCustomerData['Lost_Customers_Count'] = lostCustomerData['Lost_Customers'].str.len()
lostCustomerData

,Year,Lost_Customers,Lost_Customers_Count
0,2016,"[ zzcggtjwkj@gmail.com, cxqssgmmdd@gmail.com, ...",171710
1,2017,"[rcvcbonbsr@gmail.com, ewbzipqldt@gmail.com, ...",183687


# Total Customers Current Year

In [12]:
totalCustomerData = data.groupby('year').agg(customer_count=('customer_email', 'count'))
totalCustomerData['year'] = totalCustomerData.index
totalCustomerData.reset_index(drop=True)
totalCustomerData

,customer_count,year
year,,
2015,231294,2015
2016,204646,2016
2017,249987,2017


# Total Customers Previous Year

In [13]:
totalCustomerData['key'] = 1
totalCustomerPreviousYearData = pd.merge(totalCustomerData, totalCustomerData, on='key').drop('key', 1)
totalCustomerPreviousYearData = totalCustomerPreviousYearData.query('year_x == year_y+1')
totalCustomerPreviousYearData.drop(['customer_count_x','year_y'], axis=1, inplace=True)
totalCustomerPreviousYearData

,year_x,customer_count_y
3,2016,231294
7,2017,204646
